# Predykcja przetrwania dla pacjentów z niewydolnością serca
***

### Scenariusz
***
Jeden ze szpitali przygotował zestaw danych dotyczący pacjentów z niewydolnością serca. Na podstawie przykładów blisko 300 osób należy stworzyć model, który powinien prawidłowo przewidzieć, czy dany pacjent przeżyje. W tym celu zdecydowano się wykorzystać Azure Machine Learning w celu automaycznego doboru optymalnego rozwiązania. Poniższy notatnik ma za zadanie przejścia przez tok myślenia twórcy i w jawny sposób ukazanie prostotę wykorzystania narzędzia.

### Dane
***
Dane wykorzystane podczas przygotowywania projektu są dostępne [tutaj](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data).

Dane zawierają 13 kolumn:
> 1. age - zawiera informacje o wieku pacjenta
> 2. anaemia - określa, czy pacjent posiadał anemię czyli zmniejszoną liczbę czerwonych krwinek lub hemoglobiny
> 3. creatinine_phosphokinase - poziom enzymu CPK we krwi (mcg/L)
> 4. diabetes - określa, czy pacjent miał cukrzycę
> 5. ejection_fraction - procent krwi opuszczającej serce przy każdym skurczu
> 6. high_blood_pressure - określa, czy pacjent miał nadciśnienie
> 7. platelets - określa liczbę płytek krwi (kiloplatelets/mL)
> 8. serum_creatinine - określa poziom kreatyniny we krwi (mg/dL)
> 9. serum_sodium - określa poziom sodu we krwi (mEq/L)
> 10. sex - określa płeć pacjenta
> 11. smoking - wskazuje, czy pacjent jest palaczem
> 12. time - czas obserwacji
> 13. DEATH_EVENT - wskazuje, czy pacjent zmarł w trakcie obserwacji


### Odtworzenie rozwiązania
***
1. Stworzenie zasobu Machine Learning
2. Przejście do Machine Learning Studio
3. Załadowanie danych w zakładce Datasets
4. Dodanie pliku ipynb w zakładce Notebooks
5. Przejście do wczytanego Notebooka
6. Stworzenie nowej wirtualnej masazyny odpowiedzialnej za obliczenia (w pierwotnym przypadku - 2 cores, 4 GB RAM, 14 GB)
7. Uruchomienie całego skryptu
8. Ewentualna publikacja modelu i predykcje nowych danych

### Import wymaganych bibliotek

In [1]:
import pandas as pd
import numpy as np

from azureml.core import Workspace, Dataset, Experiment
from azureml.widgets import RunDetails
from azureml.train import automl

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

### Wczytanie przestrzeni roboczej z serwera
***
Niekiedy wymagana będzie autentykacja. W tym celu należy postępować zgodnie z wypisywanymi poleceniami.

In [2]:
ws = Workspace.from_config()

### Załadowanie do pamięci zbioru danych
***
Jest to zbiór danych dodany wcześniej w zakładce Datasets. Jego wyszukiwanie przeprowadzane jest po jego nazwie. Następnym krokiem jest przekształcenie danych do formy DataFrame z biblioteki Pandas. Dodatkowo usuwane są wiersze, w których nie mamy wszystkch informacji.

In [3]:
dataset = Dataset.get_by_name(workspace=ws, name="HeartFailure")
df = dataset.to_pandas_dataframe().dropna()

### Wypisanie podstawowych informacji dotyczących zbioru
***
Na podstawie poniższej tabeli można wnioskować, że wszystkie cechy mogą zostać wykorzystane podczas treningu, ponieważ mają wysokie odchylenia standardowe w porównaniu ze średnią. Nie widzę zatem podstawy do ręcznego usuwania cech.

In [4]:
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000
mean,60.835017,0.427609,584.898990,0.414141,38.104377,0.350168,263144.952020,1.394848,136.629630,0.649832,0.323232,129.983165,0.319865
std,11.934919,0.495567,972.837034,0.493404,11.864916,0.477828,97794.623711,1.037728,4.427059,0.477828,0.468500,77.801656,0.467211
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.500000,113.000000,0.000000,0.000000,4.000000,0.000000
25%,51.000000,0.000000,118.000000,0.000000,30.000000,0.000000,213000.000000,0.900000,134.000000,0.000000,0.000000,73.000000,0.000000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.100000,137.000000,1.000000,0.000000,113.000000,0.000000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303000.000000,1.400000,140.000000,1.000000,1.000000,205.000000,1.000000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.400000,148.000000,1.000000,1.000000,285.000000,1.000000


### Podział danych na zbiór treningowy oraz testowy
***
Zbiór treningowy zawierać będzie 80% wszystkich danych, a pozostałe dane trafią do zbioru testowego. Dodatkowo dodano parametr random_state, który pozwala na wykonywanie podziału w ten sam sposób za każdym razem. Parametr ten nie jest wymagany.

In [5]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=200)

### Wypisanie możliwych metryk oceny jakości klasyfikacji
***
Poniżej wypisano listę możliwych metryk do oceny jakości klasyfikacji. W naszej aplikacji wykrozystamy metrykę accuracy, która określona jest jako stosunek liczby prawidłowych klasyfikacji do liczby wszystkich klasyfikacji.

In [6]:
automl.utilities.get_primary_metrics('classification')

['AUC_weighted',
 'accuracy',
 'norm_macro_recall',
 'average_precision_score_weighted',
 'precision_score_weighted']

### Przygotowanie wstępnych ustawień eksperymentu
***
Ustawiamy kolejno:
- Maksymalna liczba minut, jaką może być trenowany jeden model
- Włączamy możliwość wcześniejszego przerwania treningu, jeśli nie przynosi on poprawy modelu
- Dajemy możliwość wstęnego przetwarzania danych automatycznie
- Ustalamy liczbę Foldów w walidacji krzyżowej na 5

In [17]:
automl_settings = {
    "iteration_timeout_minutes" : 2,
    "enable_early_stopping" : True,
    "featurization" : 'auto',
    "n_cross_validations" : 5
}

### Przygotowanie ustawień eksperymentu c.d.
***
Ustawiamy kolejno:
- wykonywane zadanie jako klasyfikacja
- plik do zapisywania logów
- dane treningowe
- kolumnę, która podlega klasyfikacji
- wcześniej zdefiniowane ustawienia

In [8]:
automl_config = automl.AutoMLConfig(task="classification",
                            debug_log="automl_errors.log",
                            training_data=X_train,
                            label_column_name="DEATH_EVENT",
                            **automl_settings
                            )

### Tworzenie nowego eksperymentu o zadanej nazwie

In [9]:
experiment = Experiment(ws, "HeartFailureExperiment")

### Uruchomienie eksperymentu

In [10]:
run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_b380ebbc-566a-4f05-a475-fce2f40b8161

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

**************************************************

### Wypisanie informacji o przeprowadzonym eksperymencie
***
Wnioski:
1. Przeprowadzone zostało 38 treningów różnych modeli
2. Całość treningu zajęła trochę czasu, jednak nie było to zbyt długo
3. Najwyższą dokładność osiągnął model VotingEnsemble (87.8%)
4. Najniższa dokładość została osiągnięta przez model StandardScalerWrapper, KNN (58,3%)
5. Podgląd wyników eksperymentu w ML Studio pozwala na wyciągnięcie jeszcze większej dawki informacji z przeprowadzonego eksperymentu.

In [11]:
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Wydobycie najlepszego modelu

In [12]:
best_run, best_model = run.get_output()

### Predykcja dla jednego pacjenta ze zbioru testowego
***
Poniżej przedstawiono dane pomiarowe jednego z pacjentów (63 lata, anemia itd.)
Nasz model wskazał, że pacjent nie umrze, co również potwierdzają dane zawarte w zbiorze testowym.

In [22]:
patient = X_test.drop("DEATH_EVENT", axis="columns").iloc[0]
patient

age                            63.00
anaemia                         1.00
creatinine_phosphokinase      514.00
diabetes                        1.00
ejection_fraction              25.00
high_blood_pressure             1.00
platelets                  254000.00
serum_creatinine                1.30
serum_sodium                  134.00
sex                             1.00
smoking                         0.00
time                           83.00
Name: 96, dtype: float64

In [37]:
pred = best_model.predict(pd.DataFrame([patient]))
pred

array([0])

In [38]:
X_test["DEATH_EVENT"].iloc[0]

0

### Predykcja na danych testowych
***
Przeprowadzimy teraz predykcje dla wszystkich dnaych testowych

In [39]:
preds = best_model.predict(X_test.drop("DEATH_EVENT", axis="columns"))
preds

array([0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0])

### Wypiszemy macierz pewności
***
- 41 pacjentów, zostało poprawnie zdiagnozowanych, że nie umrą
- 13 pacjentow zostało poprawnie zdiagnozowanych, że umrą
- 2 pacjentów zostało zdiagnozowanych, że nie umrą a umarli
- 4 pacjentów zostało zdiagnonowanych, że umrą a nie umarli

In [14]:
cm = confusion_matrix(X_test["DEATH_EVENT"], preds)
cm

array([[41,  2],
       [ 4, 13]])

### Wyznaczenie dokładności modelu
***
Wyznaczona dokładność wyniosła 90% (dla zbioru testowego, który nie brał udziału w procesie uczenia)

In [16]:
acc = accuracy_score(X_test["DEATH_EVENT"], preds)
print(acc)

0.9


### Wnioski
1. Udało się uzyskać bardzo dobrą jakość klasyfikacji
2. W celu wykorzystania modelu w rzeczywistym przypadku, należałoby zgromadzić więcej danych
3. Stworzenie modeli i ich trening wymagał jedynie minimalnej wiedzy z zakresu Data Science, a stworzony model cechuje się ogromną złożonością. 
4. Uzyskanie takiej dokładności ręcznie byłoby niezwykle trudne.
5. Korzystanie z Machine Learning Studio okazało się bardzo łatwe i przyjemne!